# Importando Dados

In [29]:
import pandas as pd

textos = pd.read_csv('data/ml.csv')
textos = textos[:250]
textos['label'] = textos['label'].astype('int')
print(textos.shape)
textos.head()

(250, 2)


texto  label
0  JOSÉ EGÍDIO DE SANTANA PRESIDENTE / COPEL DISP...      1
1  RA-TIFICO O ENTENDIMENTO EXPOSTO NO PARECER RE...      2
2  DISPENSO O PROCEDIMENTO LICITATÓRIO, COM BASE ...      2
3  IRANIZO MATOS RODRIGUES PREGOEIRO PORTARIA Nº ...      1
4  ANKE HELGA SCHWABE, NO USO DE SUAS ATRIBUIÇÕES...      0

In [30]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop = stopwords.words('portuguese')
print(stop)

['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estiv

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bezerraescossia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Machine Learning

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=stop)
bag = vectorizer.fit_transform(textos['texto'])

X = bag.toarray()
y = textos['label']

In [32]:
from sklearn.model_selection import train_test_split
import numpy as np

SEED = 1234
np.random.seed(SEED)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify = y)

print(f'O dataset de treino possui {X_train.shape[0]} pacientes e o de treino {X_test.shape[0]} pacientes.')

O dataset de treino possui 187 pacientes e o de treino 63 pacientes.


## Baseline

In [33]:
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(X_train, y_train)

acuracia = dummy_stratified.score(X_test, y_test) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 39.68%


## Modelos

In [34]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

modelo = LinearSVC()
modelo.fit(X_train, y_train)
previsoes = modelo.predict(X_test)

acuracia = accuracy_score(y_test, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 93.65%


/Users/bezerraescossia/Github/imd-data-science/.venv/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [35]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

modelo = SGDClassifier()
modelo.fit(X_train, y_train)
previsoes = modelo.predict(X_test)

acuracia = accuracy_score(y_test, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 92.06%


In [36]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

modelo = GaussianNB()
modelo.fit(X_train, y_train)
previsoes = modelo.predict(X_test)

acuracia = accuracy_score(y_test, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 82.54%


# Testando modelos com Validação cruzada

In [37]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

np.random.seed(SEED)
cv = StratifiedKFold(n_splits = 5, shuffle = True)

model = DecisionTreeClassifier(max_depth=3)
model_svc = SVC()
model_log = LogisticRegression(solver='liblinear')
model_rand = RandomForestClassifier(n_estimators=100)
model_linearSVC = LinearSVC()
model_sgdc = SGDClassifier()

models = [model, model_svc, model_log, model_rand, model_linearSVC, model_sgdc]
name = ['Árvore de Decisão', 'SVC', 
        'Regressão Logística', 'Random Forest', 'LinearSVC', 'SGDC']

count = 0
for item in models:
    np.random.seed(SEED)
    results = cross_val_score(item, X_train, y_train, cv = cv, 
                              scoring = 'accuracy')
    mean = results.mean()
    dv = results.std()
    print('Acurácia média - Modelo {}: {:.2f}%'.format(name[count], mean*100))
    print('Intervalo de acurácia - Modelo {}: [{:.2f}% ~ {:.2f}%]\n'.format(name[count], (mean - 2*dv)*100, (mean + 2*dv)*100))
    count += 1

Acurácia média - Modelo Árvore de Decisão: 81.74%
Intervalo de acurácia - Modelo Árvore de Decisão: [65.70% ~ 97.77%]

Acurácia média - Modelo SVC: 84.52%
Intervalo de acurácia - Modelo SVC: [78.62% ~ 90.42%]

Acurácia média - Modelo Regressão Logística: 86.63%
Intervalo de acurácia - Modelo Regressão Logística: [79.02% ~ 94.24%]

Acurácia média - Modelo Random Forest: 85.56%
Intervalo de acurácia - Modelo Random Forest: [75.55% ~ 95.57%]



/Users/bezerraescossia/Github/imd-data-science/.venv/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/bezerraescossia/Github/imd-data-science/.venv/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/bezerraescossia/Github/imd-data-science/.venv/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/bezerraescossia/Github/imd-data-science/.venv/lib/python3.10/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Acurácia média - Modelo LinearSVC: 85.02%
Intervalo de acurácia - Modelo LinearSVC: [77.75% ~ 92.29%]

Acurácia média - Modelo SGDC: 81.85%
Intervalo de acurácia - Modelo SGDC: [74.27% ~ 89.43%]

